In [1]:
import csv
import pandas as pd
import networkx as nx

In [2]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_3_only_3.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_3_only_3.csv'

In [3]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [4]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [5]:
df

,Source,Destination,Number of Tweets
0,CH,CH,37.0
1,GB,GB,2248.0
2,US,US,12773.0
3,CA,CA,802.0
4,PL,PL,50.0
...,...,...,...
599,PT,CH,1.0
600,KI,US,1.0
601,IN,PL,1.0
602,PE,SI,1.0


## Creating Graph using Networkx

In [6]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [7]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

14577.0

In [8]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [9]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,37.0,,,,
1,GB,GB,2248.0,,,,
2,US,US,12773.0,,,,
3,CA,CA,802.0,,,,
4,PL,PL,50.0,,,,
...,...,...,...,...,...,...,...
599,PT,CH,1.0,,,,
600,KI,US,1.0,,,,
601,IN,PL,1.0,,,,
602,PE,SI,1.0,,,,


In [10]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [11]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,37.0,161.0,161.0,84.0,77.0
1,GB,GB,2248.0,6215.0,6215.0,3143.0,3072.0
2,US,US,12773.0,28904.0,28904.0,14577.0,14327.0
3,CA,CA,802.0,2355.0,2355.0,1158.0,1197.0
4,PL,PL,50.0,143.0,143.0,62.0,81.0
...,...,...,...,...,...,...,...
599,PT,CH,1.0,57.0,161.0,18.0,77.0
600,KI,US,1.0,1.0,28904.0,1.0,14327.0
601,IN,PL,1.0,2126.0,143.0,1074.0,81.0
602,PE,SI,1.0,6.0,12.0,4.0,8.0


In [12]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,37.0,161.0,161.0,84.0,77.0,,
1,GB,GB,2248.0,6215.0,6215.0,3143.0,3072.0,,
2,US,US,12773.0,28904.0,28904.0,14577.0,14327.0,,
3,CA,CA,802.0,2355.0,2355.0,1158.0,1197.0,,
4,PL,PL,50.0,143.0,143.0,62.0,81.0,,
...,...,...,...,...,...,...,...,...,...
599,PT,CH,1.0,57.0,161.0,18.0,77.0,,
600,KI,US,1.0,1.0,28904.0,1.0,14327.0,,
601,IN,PL,1.0,2126.0,143.0,1074.0,81.0,,
602,PE,SI,1.0,6.0,12.0,4.0,8.0,,


In [13]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [14]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,37.0,161.0,161.0,84.0,77.0,48.051948,44.047619
1,GB,GB,2248.0,6215.0,6215.0,3143.0,3072.0,73.177083,71.524022
2,US,US,12773.0,28904.0,28904.0,14577.0,14327.0,89.153347,87.62434
3,CA,CA,802.0,2355.0,2355.0,1158.0,1197.0,67.000835,69.25734
4,PL,PL,50.0,143.0,143.0,62.0,81.0,61.728395,80.645161
...,...,...,...,...,...,...,...,...,...
599,PT,CH,1.0,57.0,161.0,18.0,77.0,1.298701,5.555556
600,KI,US,1.0,1.0,28904.0,1.0,14327.0,0.00698,100.0
601,IN,PL,1.0,2126.0,143.0,1074.0,81.0,1.234568,0.09311
602,PE,SI,1.0,6.0,12.0,4.0,8.0,12.5,25.0


## Extensions

### Tweets per population and 2 versions of Cij 

In [15]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [16]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [17]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [18]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,GB,GB,298.0,792.0,792.0,405.0,387.0,77.002584,73.580247,,,
1,US,US,1498.0,3361.0,3361.0,1692.0,1669.0,89.754344,88.534279,,,
2,IE,IE,7.0,23.0,23.0,13.0,10.0,70.0,53.846154,,,
3,CA,CA,63.0,212.0,212.0,100.0,112.0,56.25,63.0,,,
4,US,GB,69.0,3361.0,792.0,1692.0,387.0,17.829457,4.078014,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
137,IL,ZA,1.0,8.0,146.0,5.0,77.0,1.298701,20.0,,,
138,KR,IE,1.0,2.0,23.0,2.0,10.0,10.0,50.0,,,
139,BR,AU,1.0,3.0,168.0,1.0,89.0,1.123596,100.0,,,
140,US,IE,1.0,3361.0,23.0,1692.0,10.0,10.0,0.059102,,,


In [18]:
import pypopulation

In [19]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [20]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,37.0,161.0,161.0,84.0,77.0,48.051948,44.047619,0.000002,,
1,GB,GB,2248.0,6215.0,6215.0,3143.0,3072.0,73.177083,71.524022,0.000017,,
2,US,US,12773.0,28904.0,28904.0,14577.0,14327.0,89.153347,87.62434,0.000019,,
3,CA,CA,802.0,2355.0,2355.0,1158.0,1197.0,67.000835,69.25734,0.000011,,
4,PL,PL,50.0,143.0,143.0,62.0,81.0,61.728395,80.645161,0.000001,,
...,...,...,...,...,...,...,...,...,...,...,...,...
599,PT,CH,1.0,57.0,161.0,18.0,77.0,1.298701,5.555556,0.0,,
600,KI,US,1.0,1.0,28904.0,1.0,14327.0,0.00698,100.0,0.0,,
601,IN,PL,1.0,2126.0,143.0,1074.0,81.0,1.234568,0.09311,0.0,,
602,PE,SI,1.0,6.0,12.0,4.0,8.0,12.5,25.0,0.0,,


In [21]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [22]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,37.0,161.0,161.0,84.0,77.0,48.051948,44.047619,0.000002,0.114907,0.229814
1,GB,GB,2248.0,6215.0,6215.0,3143.0,3072.0,73.177083,71.524022,0.000017,0.180853,0.361706
2,US,US,12773.0,28904.0,28904.0,14577.0,14327.0,89.153347,87.62434,0.000019,0.220956,0.441911
3,CA,CA,802.0,2355.0,2355.0,1158.0,1197.0,67.000835,69.25734,0.000011,0.170276,0.340552
4,PL,PL,50.0,143.0,143.0,62.0,81.0,61.728395,80.645161,0.000001,0.174825,0.34965
...,...,...,...,...,...,...,...,...,...,...,...,...
599,PT,CH,1.0,57.0,161.0,18.0,77.0,1.298701,5.555556,0.0,0.004587,0.010526
600,KI,US,1.0,1.0,28904.0,1.0,14327.0,0.00698,100.0,0.0,0.000035,0.00007
601,IN,PL,1.0,2126.0,143.0,1074.0,81.0,1.234568,0.09311,0.0,0.000441,0.000866
602,PE,SI,1.0,6.0,12.0,4.0,8.0,12.5,25.0,0.0,0.055556,0.083333


In [23]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_3_only_3.csv', index=False)